# Delanteros

Vamos a seleccionar a los dos mejores delanteros según:

    1. Goles
    2. Asistencias
    3. Porcentaje de regates acertados 
    4. Porcentaje de pases acertados

In [1]:
import pandas as pd
import numpy as np

#leemos el csv y eliminamos duplicidades

df = pd.read_csv('/Users/alexcar934/Desktop/EDA_bien/src/data/raw/datasetMatchInfo2020-La-Liga.csv')
df = df.drop_duplicates()


In [2]:
'''Creamos un array donde aparezcan los nombres de todos los jugadores que, 
según el dataset, al menos una vez han jugado como delantero o en la línea de ataque'''
df_fw = df[df['Pos'] == 'FW']
df_fw_unique = df_fw['Player'].unique()

In [3]:
'''
Creamos un bucle for que lea todos los nombres del array para 
que les asigne la posición FW en el dataframe en todos los partidos.
'''

for i in df_fw_unique:
    if np.any(df['Player'] == i):
        df.loc[(df['Player']== i), 'Pos'] = 'FW'


/var/folders/6g/cf_j0b453g7fk51dd_1wyyzr0000gn/T/ipykernel_83643/595562217.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Pos'][df['Player']== i] = 'FW'
/var/folders/6g/cf_j0b453g7fk51dd_1wyyzr0000gn/T/ipykernel_83643/595562217.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Pos'][df['Player']== i] = 'FW'
/var/folders/6g/cf_j0b453g7fk51dd_1wyyzr0000gn/T/ipykernel_83643/595562217.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

In [4]:
'''
Para segmentar, establecemos que queremos obtener únicamente los partidos
en los que el futbolista haya jugado más de 45 minutos
'''
df_fw= df[df['Pos'] == 'FW']
df_fw =df_fw[df_fw['Min']>= 45]

In [5]:
#Creamos un dataframe con los minutos jugados por jugador
#Segmentamos por los jugadores que hayan jugado, al menos, un total de 500 minutos.
df_minutos = df_fw.groupby('Player')['Min'].sum()
df_minutos_fw = df_minutos.loc[df_minutos>500].sort_values(ascending =False)

In [6]:
#Creamos el nuevo dataframe con los futbolistas que han jugado más de 500 minutos
df_fw= df_fw[df_fw['Player'].isin(df_minutos_fw.keys())]

In [7]:

#como el mínimo de minutos es de 720 (más de 8 partidos), establezco un mínimo
#de 400 toques de balón en total.
df_toques= df_fw.groupby('Player')['Performance|Touches'].sum()
df_toques_fw = df_toques.loc[df_toques>= (400)].sort_values(ascending=False)


In [8]:
#aplico el filtro al dataframe
df_fw= df_fw[df_fw['Player'].isin(df_toques_fw.keys())]


In [9]:
'''
Creamos un dataframe con la media de recuperaciones de balón por partido.
Posteriormente los ponderamos entre el total.
'''
df_recov_try= df_fw.groupby('Player')['Performance|Recov'].mean().sort_values(ascending=False)
df_recov = df_recov_try[df_recov_try.keys().isin(df_minutos_fw.keys())]
df_recov = df_recov/df_recov.sum()

In [10]:
'''
Creamos un dataframe con las asistencias realizadas por jugador.
'''
df_ast_try= df_fw.groupby('Player')['Performance|Ast'].sum().sort_values(ascending=False)
df_ast = df_ast_try[df_ast_try.keys().isin(df_minutos_fw.keys())]

In [11]:
'''
Creamos un dataframe con la media de los porcentajes de pases acertados por partido.
Posteriormente los ponderamos entre el total.
'''
df_cmp_try= df_fw.groupby('Player')['Passes|Cmp%'].mean().sort_values(ascending=False)
df_cmp= df_cmp_try[df_cmp_try.keys().isin(df_minutos_fw.keys())]
df_cmp= df_cmp/df_cmp.sum()


In [12]:
'''
Creamos un dataframe con los goles anotados por jugador.
'''
df_gls_try= df_fw.groupby('Player')['Performance|Gls'].sum().sort_values(ascending=False)
df_gls = df_gls_try[df_gls_try.keys().isin(df_minutos_fw.keys())]

In [13]:
'''
Creamos un dataframe con la media de los porcentajes de regates acertados por partido.
Posteriormente los ponderamos entre el total.
'''
df_drbls_try= df_fw.groupby('Player')['Dribbles|Succ%'].sum().sort_values(ascending=False)
df_drbls_succ = df_drbls_try[df_drbls_try.keys().isin(df_minutos_fw.keys())]
df_drbls_succ = df_drbls_succ/(df_drbls_succ.sum())

In [14]:
#Convertimos las series a dataframe
df_drbls_succ = df_drbls_succ.to_frame()
df_recov=df_recov.to_frame()
df_cmp = df_cmp.to_frame()
df_ast = df_ast.to_frame()
df_gls = df_gls.to_frame()



In [15]:
'''
Unimos todos los dataframes por la columna Player
'''
df_fw_final = df_cmp.merge(df_ast, how='inner', on=['Player'])
df_fw_final = df_fw_final.merge(df_gls, how='inner', on=['Player'])
df_fw_final =df_fw_final.merge(df_drbls_succ, how='inner', on=['Player'])


In [16]:
df_teams= df[['Player','Team']]
df_fw=df_teams.merge(df_fw_final, how='inner', on='Player').drop_duplicates()

In [17]:
df_fw['Puntuación Final'] = (df_fw.iloc[:,2]) + (df_fw.iloc[:,3]*0.02) + (df_fw.iloc[:,4]*0.02) + (df_fw.iloc[:,5])


In [18]:
df_fw.columns = ['Jugador', 'Equipo', 'Porcentaje de pases acertados', 'Asistencias', 'Goles', 'Porcentaje de regates acertados', 'Puntuación Final']
df_fw = df_fw.sort_values(by='Puntuación Final', ascending = False)
df_fw

,Jugador,Equipo,Porcentaje de pases acertados,Asistencias,Goles,Porcentaje de regates acertados,Puntuación Final
112,Iago Aspas,Celta,0.070520,6.0,9.0,0.120118,0.490638
156,Karim Benzema,RealM,0.076694,5.0,8.0,0.090968,0.427662
25,Gerard Moreno,Villarreal,0.062258,1.0,8.0,0.090577,0.332835
170,João Félix,Atlético,0.078425,2.0,5.0,0.063001,0.281426
141,Kike,Eibar,0.063175,0.0,5.0,0.103791,0.266966
76,Lionel Messi,Barcelona,0.078430,1.0,4.0,0.060059,0.238490
98,Nolito,Celta,0.067875,1.0,3.0,0.070860,0.218735
67,Fidel,Elche,0.073471,2.0,2.0,0.045949,0.199420
37,Manu Trigueros,Villarreal,0.080473,1.0,1.0,0.074686,0.195159
14,Nabil Fekir,Betis,0.077022,2.0,1.0,0.056497,0.193519


In [19]:
df_fw.to_csv('/Users/alexcar934/Desktop/EDA_bien/src/data/processed/csv_principales/8_Delanteros.csv')